In [1]:
import torch
import time
import os
import numpy as np

from pathlib import Path
from torch.autograd import Variable
from tensorboardX import SummaryWriter

from model import MADDPG

In [2]:
from unityagents import UnityEnvironment

In [3]:
env = UnityEnvironment(file_name='./Tennis_Windows_x86_64/Tennis.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [5]:
#edit current run
curr_run = 'run6'
model_dir = Path('./models') 
run_dir = model_dir / curr_run
log_dir = run_dir / 'logs'

#os.makedirs(run_dir)
#os.makedirs(log_dir)

logger = SummaryWriter(str(log_dir))

In [6]:

# number of training episodes.
number_of_episodes = 10000
episode_length = 50
batch_size = 254

# amplitude of OU noise
# this slowly decreases to 0
noise = 1
noise_reduction = 0.9999

hidden_dim_actor = 524
hidden_dim_critic = 524
gamma=0.99
tau=0.0001
lr_actor=0.0008
lr_critic=0.0008

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


agent_init_params = {'num_in_pol': state_size,
                     'num_out_pol': action_size,
                     'num_in_critic': state_size,
                     'hidden_dim_actor': hidden_dim_actor,
                     'hidden_dim_critic': hidden_dim_critic,
                     'tau':tau,
                     'gamma': gamma,
                    'lr_actor':lr_actor,
                    'lr_critic':lr_critic,
                    'batch_size':batch_size,
                    'max_episode_len':episode_length}

init_dict = {
                     'alg_types': 'MADDPG',
                     'agent_init_params':agent_init_params,
                     'num_agents': num_agents,
                     'discrete_action': False}



In [7]:
#maddpg = MADDPG.init_from_save(run_dir / 'model2.pth')

In [8]:

maddpg = MADDPG(**init_dict)

scores1 = []
scores2 = []
mean_scores = []
maddpg.prep_rollouts(device=device)
maddpg.prep_training(device=device)

for i_episode in range(0, number_of_episodes):
    env_info = env.reset(train_mode=True)[brain_name]
    state = env_info.vector_observations
    maddpg.reset_noise()
    score1 = 0
    score2 = 0
    t_step = 0
    while True:
        torch_actions = maddpg.act(state)
        # convert actions to numpy arrays
        agent_actions = [ac.data.numpy() for ac in torch_actions]

        env_info = env.step(agent_actions)[brain_name]
        next_state = env_info.vector_observations
        reward = env_info.rewards
        done = env_info.local_done
        
        maddpg.step(state, agent_actions, reward,next_state, done,t_step)
        state = next_state
        score1 += reward[0]
        score2 += reward[1]
        t_step+=1
        
        if np.all(done):
            break
            
    scores1.append(score1)
    scores2.append(score2)
    mean_scores.append(np.mean([score1, score2]))
    mean1 = np.mean(scores1[-100:])
    mean2 = np.mean(scores2[-100:])
    mean_score = np.mean(mean_scores[-100:])
    print('\rEpisode {}\t Mean Score for Agent 1: {:.3f}, Mean Score for Agent 2: {:.3f}, Mean Score of both Agents: {:.3f}'.format(i_episode, mean1, mean2, mean_score),end="")
    
    if i_episode % 200 ==0:
        #logger.add_scalar('agent%i/mean_episode_rewards' % i_episode,  mean_score)
        print('\rEpisode {}\t Mean Score for Agent 1: {:.3f}, Mean Score for Agent 2: {:.3f}, Mean Score of both Agents: {:.3f}'.format(i_episode, mean1, mean2, mean_score))
    
    if i_episode % 300 ==0:
        maddpg.save(run_dir / 'model2.pth')
    
    maddpg.save(run_dir / 'model3.pth')
    logger.export_scalars_to_json(str(log_dir / 'summary.json'))
    logger.close()
    
    

        

Episode 0	 Mean Score for Agent 1: 0.000, Mean Score for Agent 2: -0.010, Mean Score of both Agents: -0.005
Episode 30	 Mean Score for Agent 1: 0.001, Mean Score for Agent 2: -0.007, Mean Score of both Agents: -0.003torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Size([254, 24])
Episode 31	 Mean Score for Agent 1: 0.001, Mean Score for Agent 2: -0.007, Mean Score of both Agents: -0.003torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Size([254, 24])
torch.Size([254, 1, 2])
torch.Si

KeyboardInterrupt: 